In [4]:
import cv2
import numpy as np
import os

# Define the paths to the folders
ground_truth_folder = '/Users/jamesdarby/Documents/McGill/COMP 551/Asg4/UDED/gt'
result_folder = '/Users/jamesdarby/Documents/McGill/COMP 551/Asg4/TEED/result/BIPED2UDED/fused'

# Get a list of filenames in the ground truth folder
ground_truth_images = os.listdir(ground_truth_folder)

# Loop through each file in the ground truth folder
for filename in ground_truth_images:
    if filename.endswith('.png'):
        # Construct the full file paths
        gt_path = os.path.join(ground_truth_folder, filename)
        result_path = os.path.join(result_folder, filename)

        # Read the images
        gt_image = cv2.imread(gt_path, cv2.IMREAD_UNCHANGED)
        result_image = cv2.imread(result_path, cv2.IMREAD_UNCHANGED)

        if result_image is not None:
            # Invert the image
            inverted_image = 255 - result_image

        # Check if both images are loaded
        if gt_image is not None and result_image is not None:
            # Resize result image to match ground truth image if necessary
            if gt_image.shape != result_image.shape:
                result_image = cv2.resize(result_image, (gt_image.shape[1], gt_image.shape[0]))

            inv_diff_img = cv2.absdiff(gt_image, inverted_image)

            # Process the difference here (e.g., calculate statistics or save the image)
            # For example, to save the difference image:
            # diff_path = os.path.join('path/to/difference_folder', filename)
            # cv2.imwrite(diff_path, difference)

            # To calculate and print mean difference
            inv_diff = np.mean(inv_diff_img)
            print(f"Mean difference for {filename}:  {inv_diff}")

# Add any additional processing as needed


Mean difference for 1-0843x4.png:  10.018514662502168
Mean difference for 10-lena.png:  22.375782012939453
Mean difference for 8-ADE20K-1C.png:  26.452731481481482
Mean difference for 17-1194.png:  45.323030555555555
Mean difference for 2-0868x4.png:  11.694742321707444
Mean difference for 29-img_5182.png:  16.489420168067227
Mean difference for 27-img_5264.png:  19.661710084033615
Mean difference for 6-elephant_3.png:  8.48270034790039
Mean difference for 24-2010_002838.png:  20.16920094562648
Mean difference for 20-2009_003829.png:  26.78303592814371
Mean difference for 16-P1020854.png:  21.914449869791667
Mean difference for 25-2008_002622.png:  15.015391549295774
Mean difference for 14-comic.png:  55.69631024930748
Mean difference for 13-cameraman.png:  17.549687430385386
Mean difference for 22-335094.png:  37.48322225892319
Mean difference for 15-tire.png:  16.111541748046875
Mean difference for 7-CITYSCAPES-2C.png:  17.355756172839506
Mean difference for 28-img_043_SRF_2_HR.png: 

In [9]:
# Initialize accumulators for MSE and MAE
total_mse = 0.0
total_mae = 0.0
num_images = 0

# Get a list of filenames in the ground truth folder
ground_truth_images = os.listdir(ground_truth_folder)

# Loop through each file in the ground truth folder
for filename in ground_truth_images:
    if filename.endswith('.png'):
        # Increment the image counter
        num_images += 1

        # Construct the full file paths
        gt_path = os.path.join(ground_truth_folder, filename)
        result_path = os.path.join(result_folder, filename)

        # Read the images
        gt_image = cv2.imread(gt_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
        result_image = cv2.imread(result_path, cv2.IMREAD_UNCHANGED).astype(np.float32)

        # Check if both images are loaded
        if gt_image is not None and result_image is not None:
            # Resize result image to match ground truth image if necessary
            if gt_image.shape != result_image.shape:
                result_image = cv2.resize(result_image, (gt_image.shape[1], gt_image.shape[0]))

            # Invert the result image
            inverted_image = 255 - result_image

            # Calculate squared differences and absolute differences
            squared_diff = (gt_image - inverted_image) ** 2
            absolute_diff = np.abs(gt_image - inverted_image)

            # Accumulate MSE and MAE
            total_mse += np.mean(squared_diff)
            total_mae += np.mean(absolute_diff)

# Calculate average MSE and MAE
average_mse = total_mse / num_images
average_mae = total_mae / num_images

# Normalize the average errors
average_mse_ratio = average_mse / 255 ** 2
average_mae_ratio = average_mae / 255

print(f"Average MSE Ratio over dataset: {average_mse_ratio}")
print(f"Average MAE Ratio over dataset: {average_mae_ratio}")

Average MSE Ratio over dataset: 0.06343134376401705
Average MAE Ratio over dataset: 0.09371904161241319


In [2]:
import torch
def bdcn_loss2(inputs, targets, l_weight=1.1):
    # bdcn loss modified in DexiNed

    targets = targets.long()
    mask = targets.float()
    num_positive = torch.sum((mask > 0.0).float()).float() # >0.1
    num_negative = torch.sum((mask <= 0.0).float()).float() # <= 0.1

    mask[mask > 0.] = 1.0 * num_negative / (num_positive + num_negative) #0.1
    mask[mask <= 0.] = 1.1 * num_positive / (num_positive + num_negative)  # before mask[mask <= 0.1]
    inputs= torch.sigmoid(inputs)
    cost = torch.nn.BCELoss(mask, reduction='none')(inputs, targets.float())
    cost = torch.sum(cost.float().mean((1, 2, 3))) # before sum
    return l_weight*cost

In [5]:

# Loop through each file in the ground truth folder
for filename in ground_truth_images:
    if filename.endswith('.png'):
        # Construct the full file paths
        gt_path = os.path.join(ground_truth_folder, filename)
        result_path = os.path.join(result_folder, filename)

        # Read the images
        gt_image = cv2.imread(gt_path, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.0
        result_image = cv2.imread(result_path, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.0

        # Check if both images are loaded
        if gt_image is not None and result_image is not None:
            # Resize result image to match ground truth image if necessary
            if gt_image.shape != result_image.shape:
                result_image = cv2.resize(result_image, (gt_image.shape[1], gt_image.shape[0]))

            # Convert images to PyTorch tensors
            gt_tensor = torch.tensor(gt_image).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions
            result_tensor = torch.tensor(result_image).unsqueeze(0).unsqueeze(0)

            # Calculate the bdcn loss
            loss = bdcn_loss2(result_tensor, gt_tensor)
            print(f"Loss for {filename}: {loss.item()}")

Loss for 1-0843x4.png: 0.04150024428963661
Loss for 10-lena.png: 0.08813481777906418
Loss for 8-ADE20K-1C.png: 0.11825072765350342
Loss for 17-1194.png: 0.21944889426231384
Loss for 2-0868x4.png: 0.06642170250415802
Loss for 29-img_5182.png: 0.07291970402002335
Loss for 27-img_5264.png: 0.09982876479625702
Loss for 6-elephant_3.png: 0.032409779727458954
Loss for 24-2010_002838.png: 0.09917829185724258
Loss for 20-2009_003829.png: 0.14548587799072266
Loss for 16-P1020854.png: 0.13548116385936737
Loss for 25-2008_002622.png: 0.074457548558712
Loss for 14-comic.png: 0.27993443608283997
Loss for 13-cameraman.png: 0.05600902438163757
Loss for 22-335094.png: 0.17815403640270233
Loss for 15-tire.png: 0.07857025414705276
Loss for 7-CITYSCAPES-2C.png: 0.08494852483272552
Loss for 28-img_043_SRF_2_HR.png: 0.2061677873134613
Loss for 30-167062.png: 0.035149212926626205
Loss for 4-0896x4.png: 0.06447357684373856
Loss for 11-NYUD-2.png: 0.12742844223976135
Loss for 9-MDBD-1C.png: 0.0680684298276901

In [ ]:
# ------------ cats losses ----------
def bdrloss(prediction, label, radius,device='cpu'):
    '''
    The boundary tracing loss that handles the confusing pixels.
    '''

    filt = torch.ones(1, 1, 2*radius+1, 2*radius+1)
    filt.requires_grad = False
    filt = filt.to(device)

    bdr_pred = prediction * label
    pred_bdr_sum = label * F.conv2d(bdr_pred, filt, bias=None, stride=1, padding=radius)

    texture_mask = F.conv2d(label.float(), filt, bias=None, stride=1, padding=radius)
    mask = (texture_mask != 0).float()
    mask[label == 1] = 0
    pred_texture_sum = F.conv2d(prediction * (1-label) * mask, filt, bias=None, stride=1, padding=radius)

    softmax_map = torch.clamp(pred_bdr_sum / (pred_texture_sum + pred_bdr_sum + 1e-10), 1e-10, 1 - 1e-10)
    cost = -label * torch.log(softmax_map)
    cost[label == 0] = 0

    return torch.sum(cost.float().mean((1, 2, 3)))

def textureloss(prediction, label, mask_radius, device='cpu'):
    '''
    The texture suppression loss that smooths the texture regions.
    '''
    filt1 = torch.ones(1, 1, 3, 3)
    filt1.requires_grad = False
    filt1 = filt1.to(device)
    filt2 = torch.ones(1, 1, 2*mask_radius+1, 2*mask_radius+1)
    filt2.requires_grad = False
    filt2 = filt2.to(device)

    pred_sums = F.conv2d(prediction.float(), filt1, bias=None, stride=1, padding=1)
    label_sums = F.conv2d(label.float(), filt2, bias=None, stride=1, padding=mask_radius)

    mask = 1 - torch.gt(label_sums, 0).float()

    loss = -torch.log(torch.clamp(1-pred_sums/9, 1e-10, 1-1e-10))
    loss[mask == 0] = 0

    return torch.sum(loss.float().mean((1, 2, 3)))

def cats_loss(prediction, label, l_weight=[0.,0.], device='cpu'):
    # tracingLoss

    tex_factor,bdr_factor = l_weight
    balanced_w = 1.1
    label = label.float()
    prediction = prediction.float()
    with torch.no_grad():
        mask = label.clone()

        num_positive = torch.sum((mask == 1).float()).float()
        num_negative = torch.sum((mask == 0).float()).float()
        beta = num_negative / (num_positive + num_negative)
        mask[mask == 1] = beta
        mask[mask == 0] = balanced_w * (1 - beta)
        mask[mask == 2] = 0

    prediction = torch.sigmoid(prediction)

    cost = torch.nn.functional.binary_cross_entropy(
        prediction.float(), label.float(), weight=mask, reduction='none')
    cost = torch.sum(cost.float().mean((1, 2, 3)))  # by me
    label_w = (label != 0).float()
    textcost = textureloss(prediction.float(), label_w.float(), mask_radius=4, device=device)
    bdrcost = bdrloss(prediction.float(), label_w.float(), radius=4, device=device)

    return cost + bdr_factor * bdrcost + tex_factor * textcost